## Global-based Inverse Design
---

We loaded test-set (2500 sample), the same one as we load to evaluate the forward model (02b) and inverse design (04a). In which, we define with variable name `df_test`. 

However, in this tutorial we store the first 100 samples in variable name `df_test_100`, and run the optimization only on these samples. Therefore, in order to produce results likely similar as in our paper. We recommed to use `df_test` (2500 samples) in loop process. 

`for idx, row in df_test.iterrows():`

## Import Module

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import pickle
import pymiecs
import tensorflow as tf
from tensorflow import keras
import nevergrad as ng
from concurrent import futures
import time

## Reload the data scaler

In [3]:
# preprocessor path
preprocessor_path = "datasets/scaler_particle_geometries.pkl"
scaler_Qfwd_path = "datasets/scaler_Qfwd.pkl"
scaler_Qback_path = "datasets/scaler_Qback.pkl"

# Load the preprocessors and scalers
with open(preprocessor_path, "rb") as f:
    preprocessor = pickle.load(f)
with open(scaler_Qfwd_path, "rb") as f:
    scaler_Qfwd = pickle.load(f)
with open(scaler_Qback_path, "rb") as f:
    scaler_Qback = pickle.load(f)

/home/pwiecha/.pykeras/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pwiecha/.pykeras/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pwiecha/.pykeras/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.5.2 when using version 1.6.0. 

### Mie Utils

In [4]:
# %% --- Mie
def get_Mie_spec(wavelengths, r_core, r_shell, mat_core, mat_shell, n_env):

    k0 = 2 * np.pi / wavelengths
    n_core = mat_core.get_refindex(wavelengths)
    n_shell = mat_shell.get_refindex(wavelengths)

    res = pymiecs.Q(
        k0,
        r_core=r_core,
        n_core=n_core,
        r_shell=r_shell,
        n_shell=n_shell,
        n_env=n_env.real**0.5,  # host medium must be lossless
    )
    return (
        res["qsca"],
        res["qback"],
        res["qfwd"],
    )

In [5]:
Si = pymiecs.materials.MaterialDatabase("Si")
SiO2 = pymiecs.materials.MaterialDatabase("SiO2")
Si3N4 = pymiecs.materials.MaterialDatabase("Si3N4")
Au = pymiecs.materials.MaterialDatabase("Au")
Ag = pymiecs.materials.MaterialDatabase("Ag")
ZrO2 = pymiecs.materials.MaterialDatabase("ZrO2")
TiO2 = pymiecs.materials.MaterialDatabase("TiO2")


# Define a function to map material names to material objects
def get_material(material_name):
    if material_name == "Si":
        return Si
    elif material_name == "SiO2":
        return SiO2
    elif material_name == "Au":
        return Au
    elif material_name == "Ag":
        return Ag
    elif material_name == "Si3N4":
        return Si3N4
    elif material_name == "ZrO2":
        return ZrO2
    elif material_name == "TiO2":
        return TiO2
    else:
        raise ValueError(f"Unknown material: {material_name}")

## reload Test data 

In [6]:
hdf5_df_file = "datasets/core_shell_particles_raw_122500_test_with_pred.h5"  # depend on what test set that forward model evalaute on
df_test = pd.read_hdf(hdf5_df_file)
df_test.head()  # 2500 sam

,mat_core,mat_shell,r_core,r_shell,wavelength,Q_sca,Q_back,Q_fwd,Q_fwd_pred,Q_back_pred
0,ZrO2,Si,39,132,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.038010369711505, 1.809512402100606, 1.48074...","[0.6439440129016083, 0.4467094204783908, 0.555...","[8.631606079996017, 7.795760005483272, 6.22831...","[8.745616, 7.7675796, 6.3042693, 4.196197, 3.2...","[0.6267073, 0.44084805, 0.55183196, 0.5212697,..."
1,Au,Si3N4,65,92,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.7478421540587286, 0.7347653970480106, 0.723...","[0.25254436594760143, 0.2641361829295818, 0.27...","[3.0595320722390036, 2.910348641419738, 2.7718...","[3.063077, 2.9394855, 2.7923834, 2.6702833, 2....","[0.25272343, 0.2650609, 0.28329864, 0.29937956..."
2,Si,Si,31,32,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.42059673950184884, 0.28402220125650635, 0.2...","[0.12245274773436954, 0.03833342713719669, 0.0...","[1.1619876883360172, 0.8312774845838197, 0.618...","[1.1966718, 0.8318415, 0.6197878, 0.48798603, ...","[0.16796815, 0.028995086, 0.03501954, 0.048387..."
3,ZrO2,ZrO2,81,114,"[400.0, 406.3492063492063, 412.6984126984127, ...","[4.515870402326752, 4.215809011163139, 4.05102...","[2.008253901402069, 1.6064395105460934, 1.4057...","[19.18042348996285, 16.158873919994733, 14.280...","[19.073994, 16.155634, 14.423496, 13.176519, 1...","[1.9703677, 1.5975323, 1.4016687, 1.3178806, 1..."
4,Au,TiO2,49,76,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.5633444541178934, 2.589277681420846, 2.5882...","[1.6053755830270218, 1.4718419539005771, 1.314...","[6.575478822430851, 6.83471245910305, 7.041375...","[6.792607, 6.961959, 7.1334324, 7.189859, 7.28...","[1.5677507, 1.4955047, 1.3478626, 1.1710209, 0..."


In [7]:
df_test.describe()

,r_core,r_shell
count,2500.000000,2500.000000
mean,49.083600,100.342000
std,28.653925,40.494387
min,1.000000,6.000000
25%,25.000000,72.000000
50%,49.000000,100.500000
75%,73.000000,130.000000
max,100.000000,200.000000


In [8]:
df_test["log_Qfwd"] = df_test["Q_fwd"].apply(lambda x: np.log1p(np.array(x)))
df_test["log_Qback"] = df_test["Q_back"].apply(lambda x: np.log1p(np.array(x)))

In [9]:
df_test_100 = df_test.head(100)

## Objective Function

In [10]:
def calculate_mie_scattering_error(
    r_core,
    r_shell,
    core_material,
    shell_material,
    target_Qfwd_transformed,
    target_Qback_transformed,
):
    # Define wavelengths
    wavelengths = np.linspace(400, 800, 64)

    # Get material properties using your material function
    mat_core = get_material(core_material)
    mat_shell = get_material(shell_material)

    # Calculate Mie scattering using the new package
    Qsca, Qback, Qfwd = get_Mie_spec(
        wavelengths=wavelengths,
        r_core=r_core,
        r_shell=r_shell,
        mat_core=mat_core,
        mat_shell=mat_shell,
        n_env=1.0,  # Medium refractive index
    )

    # Apply log1p transformation to Mie Qfwd and Qback
    log_mie_Qfwd = np.log1p(Qfwd)
    log_mie_Qback = np.log1p(Qback)

    # Now, apply MinMax scaling to log-transformed Mie values
    log_mie_Qfwd_transformed = scaler_Qfwd.transform(log_mie_Qfwd.reshape(1, -1))
    log_mie_Qback_transformed = scaler_Qback.transform(log_mie_Qback.reshape(1, -1))

    # Compute the MSE between the predicted and target values
    mse_Qfwd = np.mean((log_mie_Qfwd_transformed - target_Qfwd_transformed) ** 2)
    mse_Qback = np.mean((log_mie_Qback_transformed - target_Qback_transformed) ** 2)

    # Total error is the sum of MSEs
    total_error = mse_Qfwd + mse_Qback
    return total_error

In [11]:
def objective_function(
    geometry_params, target_Qfwd_transformed, target_Qback_transformed
):
    r_core_real, r_shell_real, mat_core, mat_shell = geometry_params
    total_error = calculate_mie_scattering_error(
        r_core_real,
        r_shell_real,
        mat_core,
        mat_shell,
        target_Qfwd_transformed,
        target_Qback_transformed,
    )

    return total_error

## Run global Optimization

In [12]:
# Define the search space for core radius, shell thickness, core material, shell material
parametrization = ng.p.Tuple(
    ng.p.Scalar(lower=1, upper=100),  # Real core radius (nm)
    ng.p.Scalar(lower=6, upper=200),  # Real shell thickness (nm)
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4", "ZrO2", "TiO2"]),  # Core material
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4", "ZrO2", "TiO2"]),  # Shell material
)

In [14]:
# Initialize lists to store results and runtimes
sample_runtimes = []
results = []
num_workers = 25
budget = 1000

for idx, row in df_test_100.iterrows():
    print(f"Processing sample {idx + 1}/{len(df_test_100)}")

    # Extract the target data for this sample
    target_Qfwd = np.array(row["log_Qfwd"]).reshape(1, -1)
    target_Qback = np.array(row["log_Qback"]).reshape(1, -1)

    # Apply MinMax scaling to the target
    target_Qfwd_transformed = scaler_Qfwd.transform(target_Qfwd)
    target_Qback_transformed = scaler_Qback.transform(target_Qback)
    obj_fun = lambda params: objective_function(
        params, target_Qfwd_transformed, target_Qback_transformed
    )

    # Initialize Differential Evolution optimizer with specified crossover and population size
    optim_algo = ng.families.DifferentialEvolution(
        crossover="twopoints", popsize=num_workers
    )

    optimizer = optim_algo(parametrization, budget=budget)
    start_time = time.time()

    # Use a ThreadPoolExecutor for parallel batch processing
    with futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        recommendation = optimizer.minimize(obj_fun, executor=executor)

    end_time = time.time()
    sample_runtime = end_time - start_time

    # Append runtime and best geometry to the results
    sample_runtimes.append({"Sample": idx + 1, "Runtime (seconds)": sample_runtime})
    print(f"Runtime for sample {idx}: {sample_runtime:.2f} seconds")

    best_geometry = recommendation.value
    best_loss = recommendation.loss
    print(f"Best geometry for sample {idx}: {best_geometry}")
    print(f"Best loss for sample {idx}: {best_loss}")
    print("-------------------------------------------------------")

    # Append results to the list
    result = {
        "Sample Index": idx + 1,
        "Best Geometry": best_geometry,
        "Loss": best_loss,
        "Runtime (seconds)": sample_runtime,
    }

    results.append(result)

"""
##########################################################################################
####################### Convert the runtimes into a DataFrame ############################
##########################################################################################
"""
# Convert the results and runtimes to DataFrames and save
runtime_df = pd.DataFrame(sample_runtimes)
best_df = pd.DataFrame(results)

sample_runtime_path = "runtime/global_inverse_runtime.pkl"

with open(sample_runtime_path, "wb") as f:
    pickle.dump(runtime_df, f)

Processing sample 1/100
Runtime for sample 0: 11.47 seconds
Best geometry for sample 0: (14.871197752626482, 128.59274000724213, 'Si3N4', 'Si')
Best loss for sample 0: 0.11176236773184278
-------------------------------------------------------
Processing sample 2/100
Runtime for sample 1: 11.25 seconds
Best geometry for sample 1: (66.19525259969829, 86.87228494151131, 'SiO2', 'Au')
Best loss for sample 1: 0.006858830464549185
-------------------------------------------------------
Processing sample 3/100
Runtime for sample 2: 9.43 seconds
Best geometry for sample 2: (52.052081660435874, 30.654232111435203, 'TiO2', 'Si3N4')
Best loss for sample 2: 0.000520202014610412
-------------------------------------------------------
Processing sample 4/100
Runtime for sample 3: 12.06 seconds
Best geometry for sample 3: (20.09904129006054, 113.85836545883264, 'ZrO2', 'ZrO2')
Best loss for sample 3: 1.2361518345096868e-05
-------------------------------------------------------
Processing sample 5/1

In [15]:
# Split the 'Best Geometry' column into multiple columns
best_df[["r_core", "r_shell", "mat_core", "mat_shell"]] = pd.DataFrame(
    best_df["Best Geometry"].tolist(), index=best_df.index
)

# Drop the 'Best Geometry' column if you no longer need it
best_df = best_df.drop(columns=["Best Geometry"])
best_df = best_df[
    [
        "mat_core",
        "mat_shell",
        "r_core",
        "r_shell",
        "Sample Index",
        "Runtime (seconds)",
        "Loss",
    ]
]

In [16]:
best_df.head()

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss
0,Si3N4,Si,14.871198,128.592740,1,11.473748,0.111762
1,SiO2,Au,66.195253,86.872285,2,11.250545,0.006859
2,TiO2,Si3N4,52.052082,30.654232,3,9.432516,0.000520
3,ZrO2,ZrO2,20.099041,113.858365,4,12.056848,0.000012
4,Au,Si,72.353861,84.838776,5,12.359925,0.124807


## use Mie to calculate Qfwd and Qback

In [17]:
wavelengths = np.linspace(400, 800, 64)  # From 400 nm to 800 nm
n_env = 1.0
mie_Qfwd_list, mie_Qback_list = [], []


for idx, row in best_df.iterrows():
    mat_core = get_material(row["mat_core"])
    mat_shell = get_material(row["mat_shell"])
    r_core = row["r_core"]
    r_shell = row["r_shell"]

    # Calculate the Mie spectrum for this geometry
    _, Qback, Qfwd = get_Mie_spec(
        wavelengths, r_core, r_shell, mat_core, mat_shell, n_env
    )

    mie_Qfwd_list.append(Qfwd)
    mie_Qback_list.append(Qback)

# Convert the Mie Qfwd and Qback to DataFrame or arrays if needed
mie_Qfwd_array = np.array(mie_Qfwd_list)
mie_Qback_array = np.array(mie_Qback_list)

# Validation: Check dimensions
assert len(mie_Qfwd_array) == len(best_df), "Mismatch in dimensions for Qfwd!"
assert len(mie_Qback_array) == len(best_df), "Mismatch in dimensions for Qback!"

# Add the Mie results to the DataFrame
best_df["mie_Qfwd"] = mie_Qfwd_array.tolist()
best_df["mie_Qback"] = mie_Qback_array.tolist()

best_df

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss,mie_Qfwd,mie_Qback
0,Si3N4,Si,14.871198,128.592740,1,11.473748,1.117624e-01,"[8.533829844460708, 9.340502075444443, 8.01940...","[0.8549934083899265, 0.18965925201335165, 0.28..."
1,SiO2,Au,66.195253,86.872285,2,11.250545,6.858830e-03,"[2.88995419429865, 2.7711695656998354, 2.65987...","[0.31796077742157647, 0.33129509976661764, 0.3..."
2,TiO2,Si3N4,52.052082,30.654232,3,9.432516,5.202020e-04,"[0.6381934643532078, 0.5824347586518034, 0.532...","[0.027964889052919984, 0.030175995877572713, 0..."
3,ZrO2,ZrO2,20.099041,113.858365,4,12.056848,1.236152e-05,"[18.90946262780809, 15.986045111765787, 14.169...","[1.9714889761353533, 1.587385404767331, 1.3969..."
4,Au,Si,72.353861,84.838776,5,12.359925,1.248069e-01,"[3.428719735337389, 3.6685573523666233, 4.1377...","[0.8929630365246797, 0.8924259681667782, 0.984..."
...,...,...,...,...,...,...,...,...,...
95,ZrO2,SiO2,27.619355,110.352330,96,10.778603,5.915117e-06,"[4.9927770217688305, 4.63655206688677, 4.30918...","[0.0535793881702556, 0.05885103164005294, 0.06..."
96,TiO2,SiO2,4.797619,139.131046,97,10.763443,2.025182e-06,"[10.930652948427614, 10.409979899523954, 9.923...","[0.42602207638672573, 0.39620077227757344, 0.3..."
97,Si,Ag,65.918305,92.601877,98,11.777781,6.748079e-03,"[12.098227340175715, 11.519809044929122, 9.169...","[0.18736575196198757, 0.4097973089989122, 0.73..."
98,Au,Au,54.099457,146.001964,99,10.475445,1.192223e-09,"[13.193900418184164, 12.83603677909017, 12.502...","[0.8632642703588491, 0.9085032748429761, 0.939..."


## Also compare Mie to foward model predictions

### Reload the forward 

#### Define the own resblock class

keras requires custom classes to be defined for being able to reload

In [18]:
@keras.utils.register_keras_serializable()
class ResBlock1D(keras.Model):
    def __init__(self, filters, kernel_size=3, convblock=False, **kwargs):
        super(ResBlock1D, self).__init__(**kwargs)

        # setup all necessary layers
        self.conv1 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn1 = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()

        self.relu = keras.layers.LeakyReLU(negative_slope=0.1)

        self.convblock = convblock
        if self.convblock:
            self.conv_shortcut = keras.layers.Conv1D(filters, 1)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        # add shortcut. optionally pass it through a Conv
        if self.convblock:
            x_sc = self.conv_shortcut(input_tensor)
        else:
            x_sc = input_tensor
        x += x_sc
        return self.relu(x)

    def get_config(self):
        base_config = super().get_config()
        return {
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "convblock": self.convblock,
            **base_config,
        }

In [19]:
forward_path = "models/resnet_Mie_predictor.keras"
forward_model = keras.models.load_model(forward_path)

I0000 00:00:1742377959.151855 3043952 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20320 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:21:00.0, compute capability: 8.9


In [20]:
"""
#########################################################################################
############### Use forward model to predict on the optimized geometries  ###############
#########################################################################################
"""

# Ensure that the columns are ordered the same way as the input during training
categorical_features = [
    "mat_core",
    "mat_shell",
]  # Update these names based on your data
numerical_features = ["r_core", "r_shell"]

# Use the preprocessor to transform the data
X_optimized_preprocessed = preprocessor.transform(
    best_df[categorical_features + numerical_features]
)

# Predict using the forward model
y_pred = forward_model.predict(X_optimized_preprocessed)

# Separate predictions for Qfwd and Qback
y_pred_Qfwd = y_pred[..., 0]  # First output for Qfwd predictions
y_pred_Qback = y_pred[..., 1]  # Second output for Qback predictions

# Inverse transform the predicted Qfwd and Qback to get the original scale
predicted_Qfwd_original = scaler_Qfwd.inverse_transform(y_pred_Qfwd)
predicted_Qback_original = scaler_Qback.inverse_transform(y_pred_Qback)

# Apply expm1 to reverse the log1p transformation
predicted_Qfwd_original = np.expm1(predicted_Qfwd_original) 
predicted_Qback_original = np.expm1(predicted_Qback_original)  

# Display the predicted Qfwd and Qback in original scale
print("Predicted Qfwd (Original Scale):", predicted_Qfwd_original)
print("Predicted Qback (Original Scale):", predicted_Qback_original)

# Add the predicted values (in original scale) to the DataFrame
best_df["predicted_Qfwd"] = predicted_Qfwd_original.tolist()
best_df["predicted_Qback"] = predicted_Qback_original.tolist()

I0000 00:00:1742377963.212716 3050466 service.cc:148] XLA service 0x79d42000a860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742377963.212759 3050466 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-19 10:52:43.288861: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742377963.428867 3050466 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/4 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step

I0000 00:00:1742377964.217261 3050466 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 527ms/step
Predicted Qfwd (Original Scale): [[ 9.796821    7.876669    7.010841   ...  4.6654005   4.276488
   4.052196  ]
 [ 4.3910704   3.9271126   3.1057045  ...  2.0592787   2.0180721
   2.7877114 ]
 [ 1.570462    1.2410283   1.4790552  ...  0.27261904  0.3859349
   0.2648519 ]
 ...
 [12.464979    9.787121    8.414065   ...  1.1448581   1.1742796
   1.4374583 ]
 [13.8961525  12.270571   10.244209   ...  3.894843    4.0128183
   4.3889585 ]
 [10.084749    8.299186    7.3488336  ...  2.9977357   2.8558326
   2.8243735 ]]
Predicted Qback (Original Scale): [[0.8263309  1.1199386  0.99718577 ... 5.5092134  5.5265093  5.846613  ]
 [0.980454   1.3364712  1.031034   ... 2.4313202  2.5990705  2.1042504 ]
 [1.5653567  1.3143272  1.7218399  ... 0.09388033 0.6982077  0.27284992]
 ...
 [1.4000951  1.9882244  1.626818   ... 1.5801059  1.856684   1.3684772 ]
 [1.4452343  1.9294709  1.5414227  ... 3.8397937  4.0566797  3.9665504 ]
 [1.0236554  1.4966857  1.2533314  ... 

In [21]:
best_df["log_Qfwd"] = best_df["mie_Qfwd"].apply(lambda x: np.log1p(np.array(x)))
best_df["log_Qback"] = best_df["mie_Qback"].apply(lambda x: np.log1p(np.array(x)))
best_df.head()

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss,mie_Qfwd,mie_Qback,predicted_Qfwd,predicted_Qback,log_Qfwd,log_Qback
0,Si3N4,Si,14.871198,128.592740,1,11.473748,0.111762,"[8.533829844460708, 9.340502075444443, 8.01940...","[0.8549934083899265, 0.18965925201335165, 0.28...","[9.796820640563965, 7.876668930053711, 7.01084...","[0.8263309001922607, 1.119938611984253, 0.9971...","[2.254846509388286, 2.3360684245208336, 2.1993...","[0.617881142624474, 0.1736669232735747, 0.2501..."
1,SiO2,Au,66.195253,86.872285,2,11.250545,0.006859,"[2.88995419429865, 2.7711695656998354, 2.65987...","[0.31796077742157647, 0.33129509976661764, 0.3...","[4.391070365905762, 3.927112579345703, 3.10570...","[0.9804540276527405, 1.3364711999893188, 1.031...","[1.3583973823164632, 1.3273851829772618, 1.297...","[0.27608567647329674, 0.28615222766072146, 0.2..."
2,TiO2,Si3N4,52.052082,30.654232,3,9.432516,0.000520,"[0.6381934643532078, 0.5824347586518034, 0.532...","[0.027964889052919984, 0.030175995877572713, 0...","[1.5704619884490967, 1.2410283088684082, 1.479...","[1.565356731414795, 1.3143272399902344, 1.7218...","[0.4935940885621718, 0.45896464742603416, 0.42...","[0.02758101183190901, 0.02972965742901675, 0.0..."
3,ZrO2,ZrO2,20.099041,113.858365,4,12.056848,0.000012,"[18.90946262780809, 15.986045111765787, 14.169...","[1.9714889761353533, 1.587385404767331, 1.3969...","[17.779001235961914, 13.255891799926758, 11.04...","[2.73857045173645, 2.626329183578491, 2.168610...","[2.991195127646979, 2.8323921311747187, 2.7192...","[1.0890631659524277, 0.9506478695981189, 0.874..."
4,Au,Si,72.353861,84.838776,5,12.359925,0.124807,"[3.428719735337389, 3.6685573523666233, 4.1377...","[0.8929630365246797, 0.8924259681667782, 0.984...","[5.033417224884033, 3.9798107147216797, 3.9398...","[1.1223475933074951, 1.3606879711151123, 0.999...","[1.4881105434936732, 1.5408501058329949, 1.636...","[0.6381433456364796, 0.6378595870262869, 0.685..."


## Save best geometries data 

In [22]:
best_geometries_path = "best_geometries/global_inverse_test_data.pkl"

# Saving DataFrames
with open(best_geometries_path, "wb") as f:
    pickle.dump(best_df, f)